In [11]:
from llama_index.core.tools import FunctionTool
from llama_index.core import VectorStoreIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb, os
from dotenv import find_dotenv, load_dotenv
from llama_index.tools.google import GmailToolSpec

#### Function Tool

In [2]:
def get_weather(location: str) -> str:
    """Useful for getting the weather for a given location."""
    print(f"Getting weather for {location}")
    return f"The weather in {location} is sunny"

In [3]:
function_tool = FunctionTool.from_defaults(
    get_weather,
    name="my_weather_tool",
    description="Useful for getting the weather for a given location.",
)

In [4]:
function_tool_output = function_tool.call("Medellin")
function_tool_output

Getting weather for Medellin


ToolOutput(blocks=[TextBlock(block_type='text', text='The weather in Medellin is sunny')], tool_name='my_weather_tool', raw_input={'args': ('Medellin',), 'kwargs': {}}, raw_output='The weather in Medellin is sunny', is_error=False)

In [5]:
function_tool_output.raw_output

'The weather in Medellin is sunny'

---

#### Query Engine Tool

Create a **QueryEngineTool** from a **QueryEngine**

In [6]:
load_dotenv(find_dotenv())
hf_token = os.getenv("HF_TOKEN")

embed_model = HuggingFaceEmbedding("BAAI/bge-small-en-v1.5")

db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("nlp_papers")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)

llm = HuggingFaceInferenceAPI(
    model_name="Qwen/Qwen2.5-Coder-32B-Instruct",
    temperature=0.7,
    max_tokens=1000,
    token=hf_token,
    provider="auto"
)
query_engine = index.as_query_engine(llm=llm)

In [7]:
query_engine_tool = QueryEngineTool.from_defaults(query_engine, name="my_nlp_research_tool", 
                                                  description="Useful for getting information about Kevin's NLP research")

In [8]:
query_engine_tool_outut = query_engine_tool.call("What is the main contribution of Kevin Martinez to NLP?")
query_engine_tool_outut

ToolOutput(blocks=[TextBlock(block_type='text', text="Kevin Martinez's main contribution, as detailed in the provided context, involves developing and evaluating various models for detecting fake news in Spanish. This includes employing classical machine learning techniques such as SVM, Random Forest, Gradient Boosting Tree, and Multi-Layer Perceptron, as well as deep learning methods like LSTM and CNN. The work encompasses preprocessing steps like text normalization, tokenization, stemming, and removal of stopwords, as well as different text representation methods such as Bag of Words, TF-IDF, and Word Embeddings. His research also explores the use of pre-trained embeddings and the application of transfer learning, highlighting the translation of models trained on English datasets to perform effectively on Spanish datasets.")], tool_name='my_nlp_research_tool', raw_input={'input': 'What is the main contribution of Kevin Martinez to NLP?'}, raw_output=Response(response="Kevin Martinez'

In [9]:
query_engine_tool_outut.raw_output.response

"Kevin Martinez's main contribution, as detailed in the provided context, involves developing and evaluating various models for detecting fake news in Spanish. This includes employing classical machine learning techniques such as SVM, Random Forest, Gradient Boosting Tree, and Multi-Layer Perceptron, as well as deep learning methods like LSTM and CNN. The work encompasses preprocessing steps like text normalization, tokenization, stemming, and removal of stopwords, as well as different text representation methods such as Bag of Words, TF-IDF, and Word Embeddings. His research also explores the use of pre-trained embeddings and the application of transfer learning, highlighting the translation of models trained on English datasets to perform effectively on Spanish datasets."

---

#### Toolspecs

In [14]:
tool_spec = GmailToolSpec()
tool_spec_list = tool_spec.to_tool_list()
tool_spec_list

In [16]:
for tool in tool_spec_list:
    print(f"Tool: {tool.metadata.name} | Description: {tool.metadata.description}\n")

Tool: load_data | Description: load_data() -> List[llama_index.core.schema.Document]
Load emails from the user's account.

Tool: search_messages | Description: search_messages(query: str, max_results: Optional[int] = None)
Searches email messages given a query string and the maximum number
:param query: The user's query
:param max_results: The maximum number of search results

Tool: create_draft | Description: create_draft(to: Optional[List[str]] = None, subject: Optional[str] = None, message: Optional[str] = None) -> str
Create and insert a draft email.
:param to: The email addresses to send the message to
:param subject: The subject for the event
:param message: The message for the event

Tool: update_draft | Description: update_draft(to: Optional[List[str]] = None, subject: Optional[str] = None, message: Optional[str] = None, draft_id: str = None) -> str
Update a draft email.
:param to: The email addresses to send the message to
:param subject: The subject for the event
:param messa